## Bayesian Opt

In [1]:
!pip install bayesian-optimization
!pip install catboost

In [2]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import lightgbm
from bayes_opt import BayesianOptimization
from catboost import CatBoostClassifier, cv, Pool

In [3]:
train_df = pd.read_csv('flight_delays_train.csv')
test_df = pd.read_csv('flight_delays_test.csv')

train_df = train_df[train_df.DepTime <= 2400].copy() # departure time 을 기준으로 자름. 24시간 기준인데 이것을 넘어가는 것이 있어서.
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values # delay가 15분 이상된 것(Y)는 1로, 아닌 것(N)은 0으로 정답을 삼아 문제를 설계

In [4]:
train_df

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y
...,...,...,...,...,...,...,...,...,...
99995,c-5,c-4,c-3,1618,OO,SFO,RDD,199,N
99996,c-1,c-18,c-3,804,CO,EWR,DAB,884,N
99997,c-1,c-24,c-2,1901,NW,DTW,IAH,1076,N
99998,c-4,c-27,c-4,1515,MQ,DFW,GGG,140,N


In [5]:
def label_enc(df_column):
    df_column = LabelEncoder().fit_transform(df_column)
    return df_column

def make_features_sin(value, period=2400): # 주기성을 띄는 것을 sin 함수로 전처리. ex: 시간
    value *= 2 * np.pi / period 
    return np.sin(value)

def make_features_cos(value, period=2400): # 주기성을 띄는 것을 cos 함수로 전처리. ex: 시간
    value *= 2 * np.pi / period 
    return np.cos(value)

def feature_engineering(df):
    df['flight'] = df['Origin']+df['Dest'] # 출발지 + 목적지
    df['Month'] = df.Month.map(lambda x: x.split('-')[-1]).astype('int32') # '-' 로 쪼개고 마지막에 있는 것만 사용, 정수형으로 변환
    df['DayofMonth'] = df.DayofMonth.map(lambda x: x.split('-')[-1]).astype('uint8') # '-' 로 쪼개고 마지막에 있는 것만 사용, 정수형으로 변환
    df['is_begin_of_month'] = (df['DayofMonth'] < 10).astype('uint8') # 월 초
    df['is_midddle_of_month'] = ((df['DayofMonth'] >= 10)&(df['DayofMonth'] < 20)).astype('uint8')
    df['is_end_of_month'] = (df['DayofMonth'] >= 20).astype('uint8') # 월 말
    df['DayOfWeek'] = df.DayOfWeek.map(lambda x: x.split('-')[-1]).astype('uint8') # 일주일 중 몇번째 날인지, 요일 -> 숫자버젼
    df['hour'] = df.DepTime.map(lambda x: x/100).astype('int32') # 출발 시간
    df['is_morning'] = df['hour'].map(lambda x: 1 if (x <= 11)& (x >= 7) else 0).astype('uint8') # 시간대 구분 -> 단순히 시간으로 안하고 시간대로 구분한다? -> continuous하지 않은 feature. why? 단순 비례/반비례 관계 외의 정보
    df['is_daytime'] = df['hour'].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype('uint8') # 시간대 구분
    df['is_evening'] = df['hour'].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype('uint8') # 시간대 구분
    df['is_night'] = df['hour'].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype('int32') # 시간대 구분
    df['is_winter'] = df['Month'].map(lambda x: x in [12, 1, 2]).astype('int32') # 계절 구분
    df['is_spring'] = df['Month'].map(lambda x: x in [3, 4, 5]).astype('int32') # 계절 구분
    df['is_summer'] = df['Month'].map(lambda x: x in [6, 7, 8]).astype('int32') # 계절 구분
    df['is_autumn'] = df['Month'].map(lambda x: x in [9, 10, 11]).astype('int32') # 계절 구분
    df['is_holiday'] = (df['DayOfWeek'] >= 5).astype(int) # 주말
    df['is_weekday'] = (df['DayOfWeek'] < 5).astype(int) # 주중
    df['airport_dest_count'] = df.groupby(['Dest'])['Dest'].transform('count') # 해당 목적지를 향하는 총 비행편 수
    df['airport_origin_count'] = df.groupby(['Origin'])['Origin'].transform('count') # 해당 출발지로부터의 총 비행편 수
    df['carrier_count'] = df.groupby(['UniqueCarrier'])['Dest'].transform('count') # 항공/운송업체 코드 (ex AA: American Airline) 의 목적지를 향하는 총 비행편 수
    df['airport_dest_per_month'] = df.groupby(['Dest', 'Month'])['Dest'].transform('count') # 해당 목적지를 향하는 비행편의 월 편 수
    df['airport_origin_per_month'] = df.groupby(['Origin', 'Month'])['Origin'].transform('count') # 해당 출발지로부터의 비행편의 월 편 수
    df['carrier_count_per month'] = df.groupby(['UniqueCarrier', 'Month'])['Dest'].transform('count') # 항공 코드의 목적지를 향하는 비행편의 월 편 수
    df['deptime_sin'] = df['DepTime'].map(make_features_sin) # 시간은 주기성이 있기에 sin으로 처리 (2400을 주기성으로, 근데 전처리에서 이미 해주긴 했음.)
    df['deptime_cos'] = df['DepTime'].map(make_features_cos) # 시간은 주기성이 있기에 sin으로 처리 
    df['flightUC'] = df['flight']+df['UniqueCarrier'] # 출발지 + 목적지 + 항공코드
    df['DestUC'] = df['Dest']+df['UniqueCarrier'] # 목적지 + 항공코드
    df['OriginUC'] = df['Origin']+df['UniqueCarrier'] # 출발지 + 항공코드
    return df.drop('DepTime', axis=1) # depature time은 제외하고 feature engineering한 것을 return

In [6]:
train_df['Origin'].value_counts()

ATL    5822
ORD    4870
DFW    4270
LAX    3259
IAH    3048
       ... 
VIS       1
ADK       1
ILG       1
VCT       1
WYS       1
Name: Origin, Length: 289, dtype: int64

In [7]:
train_df[train_df['Origin']=='ATL']

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
21,c-6,c-21,c-3,1519,EV,ATL,ILM,377,Y
24,c-12,c-31,c-6,1837,DL,ATL,MSP,906,N
33,c-12,c-3,c-7,1653,EV,ATL,MYR,317,Y
58,c-9,c-5,c-2,1831,FL,ATL,MSP,906,Y
...,...,...,...,...,...,...,...,...,...
99911,c-7,c-4,c-1,1055,EV,ATL,CRW,363,N
99936,c-5,c-19,c-4,849,DL,ATL,SAT,874,N
99971,c-7,c-31,c-1,2027,DL,ATL,SJC,2116,Y
99981,c-8,c-15,c-1,1738,DL,ATL,PDX,2172,N


In [8]:
train_df.groupby(['Origin'])['Origin'].transform('count')

0        5822
1         688
2         868
3        2973
4        1366
         ... 
99995    1839
99996    2196
99997    1863
99998    4270
99999    1526
Name: Origin, Length: 99983, dtype: int64

In [9]:
full_df = pd.concat([train_df.drop('dep_delayed_15min', axis=1), test_df])
full_df = feature_engineering(full_df) 

for column in ['UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']: # categorical feature는 label encoding
    full_df[column] = label_enc(full_df[column])

X_train = full_df[:train_df.shape[0]]
X_test = full_df[train_df.shape[0]:]

In [10]:
X_train.head()

,Month,DayofMonth,DayOfWeek,UniqueCarrier,Origin,Dest,Distance,flight,is_begin_of_month,is_midddle_of_month,...,airport_origin_count,carrier_count,airport_dest_per_month,airport_origin_per_month,carrier_count_per month,deptime_sin,deptime_cos,flightUC,DestUC,OriginUC
0,8,21,7,1,19,82,732,171,0,0,...,11375,18024,746,1016,1569,-0.939094,0.343660,265,494,67
1,4,20,3,19,226,180,834,3986,0,0,...,1390,13069,313,105,1094,-0.790155,-0.612907,6907,1085,1441
2,9,2,5,21,239,62,416,4091,1,0,...,1747,11737,166,136,977,-0.549023,-0.835807,7064,359,1518
3,11,25,6,16,81,184,872,1304,0,0,...,6222,15343,136,514,1242,0.465615,-0.884988,2258,1122,484
4,10,7,6,20,182,210,423,2979,1,0,...,2571,30958,48,226,2674,-0.997314,0.073238,5144,1313,1103


In [11]:
from sklearn.model_selection import train_test_split

In [12]:
categorical_features = ['Month',  'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.33)

In [13]:
import lightgbm as lgb
%time 

model = lgb.LGBMClassifier()
model.fit(X_train, y_train, categorical_feature=categorical_features)
model.score(X_valid, y_valid)

CPU times: user 5 µs, sys: 4 µs, total: 9 µs
Wall time: 16.2 µs


0.8094862858008789

In [14]:
from skopt import BayesSearchCV

%time

opt = BayesSearchCV(
    estimator = lgb.LGBMClassifier(),
    search_spaces= {
        'num_leaves': (15, 50), # [15, 25, 50]
        'lambda_l2': (0.0, 0.05),
        'lambda_l1': (0.0, 0.05),
        'min_child_samples': (50, 10000),
        'min_data_in_leaf': (100, 2000)
    },
    n_iter=32,
    cv=3
)

opt.fit(X_train, y_train)
opt.score(X_valid, y_valid)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.87 µs


0.8189119563570237

In [15]:
print(opt.best_params_)

OrderedDict([('lambda_l1', 0.03955250913818615), ('lambda_l2', 0.04130022508480205), ('min_child_samples', 7885), ('min_data_in_leaf', 100), ('num_leaves', 50)])


In [16]:
opt.score(X_valid, y_valid)

0.8189119563570237